# Introduction

This Jupyter Notebook implements a comparison of different reinforcement learning methods and a value iteration method for a given M/M/c queueing system. The implemented methods include:

- Value iteration
- Tabular Q-learning
- SARSA
- Deep Q-learning
- A2C

The performance of the resulting policy of the methods are compared using simulation. The system is simulated over several replications and the expected value of the number of items in the system is reported.

# Import required packages

In [1]:
# import the algorithms from controlledmmcqueue package
from controlledmmcqueue import MMCEnv, ValueIteration, TabularQLearner, SARSA, DQNLearner, A2CLearner

# import matplotlib for plotting
import matplotlib.pyplot as plt

import numpy as np

# Build the M/M/c environment

Before running the algorithms, the M/M/c environment needs to be built. The environment is based on the `gym.Env`.

In [2]:
# queueing system parameters
arrival_rate = 2.0
service_rates = [2.0, 0.1]
queue_capacity = 100

# maximum events to use as the termination condition for each episode of the environment
max_events_per_episode = 2000

env = MMCEnv(arr_rate=arrival_rate,
             srv_rates=service_rates, queue_cap=queue_capacity,
             punish_reward=-99999,
             scale_rewards=False,
             max_events=max_events_per_episode)

# Run the algorithms to learn the optimal Policy

## Value iteration

Value iteration is implemented to compute the optimal policy. The transition probabilities of the system are known to the algorithm. The result of value iteration can be used as a benchmark.

In [3]:
# create an instance of the learner
learner_val_iter = ValueIteration(mmc_env=env, discount_factor=0.99)

# run the algorithm to learn the optimal policy
learner_val_iter.learn(convergence_threshold=1e-5, log_progress=True)

2022-08-04 12:00:04.607   learners   [inf] [LEARN-ValueIteration]   iter=     1, max_delta_v= 102.00000000
2022-08-04 12:00:04.620   learners   [inf] [LEARN-ValueIteration]   iter=     2, max_delta_v= 100.47292683
2022-08-04 12:00:04.635   learners   [inf] [LEARN-ValueIteration]   iter=     3, max_delta_v=  99.19999649
2022-08-04 12:00:04.655   learners   [inf] [LEARN-ValueIteration]   iter=     4, max_delta_v=  97.94810883
2022-08-04 12:00:04.666   learners   [inf] [LEARN-ValueIteration]   iter=     5, max_delta_v=  96.76725149
2022-08-04 12:00:04.677   learners   [inf] [LEARN-ValueIteration]   iter=     6, max_delta_v=  95.60334505
2022-08-04 12:00:04.693   learners   [inf] [LEARN-ValueIteration]   iter=     7, max_delta_v=  94.48088525
2022-08-04 12:00:04.709   learners   [inf] [LEARN-ValueIteration]   iter=     8, max_delta_v=  93.37260545
2022-08-04 12:00:04.725   learners   [inf] [LEARN-ValueIteration]   iter=     9, max_delta_v=  92.29429590
2022-08-04 12:00:04.737   learners   

### Optimal policy when the fast server is busy

We know that the optimal policy for an M/M/2 system, in case the fast server is busy is of a threshold policy based on the number of items waiting in the queue. Let's check if it is the case in the optimal policy learned by value iteration.

In [4]:
fast_server = np.argmax(service_rates)
slow_server = np.argmin(service_rates)

for n_in_queue in range(1, queue_capacity // 2):
    state = [n_in_queue] + [1 if s == fast_server else 0 for s in range(len(service_rates))]
    print(f"Optimal policy for n_in_queue = {n_in_queue:3d}: {learner_val_iter.predict(state)}")

Optimal policy for n_in_queue =   1: ()
Optimal policy for n_in_queue =   2: ()
Optimal policy for n_in_queue =   3: ()
Optimal policy for n_in_queue =   4: ()
Optimal policy for n_in_queue =   5: (2,)
Optimal policy for n_in_queue =   6: (2,)
Optimal policy for n_in_queue =   7: (2,)
Optimal policy for n_in_queue =   8: (2,)
Optimal policy for n_in_queue =   9: (2,)
Optimal policy for n_in_queue =  10: (2,)
Optimal policy for n_in_queue =  11: (2,)
Optimal policy for n_in_queue =  12: (2,)
Optimal policy for n_in_queue =  13: (2,)
Optimal policy for n_in_queue =  14: (2,)
Optimal policy for n_in_queue =  15: (2,)
Optimal policy for n_in_queue =  16: (2,)
Optimal policy for n_in_queue =  17: (2,)
Optimal policy for n_in_queue =  18: (2,)
Optimal policy for n_in_queue =  19: (2,)
Optimal policy for n_in_queue =  20: (2,)
Optimal policy for n_in_queue =  21: (2,)
Optimal policy for n_in_queue =  22: (2,)
Optimal policy for n_in_queue =  23: (2,)
Optimal policy for n_in_queue =  24: (2,)


As can be seen, the optimal policy is not to route to the slow server, unless more than 4 items are waiting in the queue.

### Optimal policy when both servers are available

In [5]:
fast_server = np.argmax(service_rates)
slow_server = np.argmin(service_rates)

for n_in_queue in range(1, queue_capacity // 2):
    state = [n_in_queue] + [0 for s in range(len(service_rates))]
    print(f"Optimal policy for n_in_queue = {n_in_queue:3d}: {learner_val_iter.predict(state)}")

Optimal policy for n_in_queue =   1: (1,)
Optimal policy for n_in_queue =   2: (1,)
Optimal policy for n_in_queue =   3: (1,)
Optimal policy for n_in_queue =   4: (1,)
Optimal policy for n_in_queue =   5: (1,)
Optimal policy for n_in_queue =   6: (1, 2)
Optimal policy for n_in_queue =   7: (1, 2)
Optimal policy for n_in_queue =   8: (1, 2)
Optimal policy for n_in_queue =   9: (1, 2)
Optimal policy for n_in_queue =  10: (1, 2)
Optimal policy for n_in_queue =  11: (1, 2)
Optimal policy for n_in_queue =  12: (1, 2)
Optimal policy for n_in_queue =  13: (1, 2)
Optimal policy for n_in_queue =  14: (1, 2)
Optimal policy for n_in_queue =  15: (1, 2)
Optimal policy for n_in_queue =  16: (1, 2)
Optimal policy for n_in_queue =  17: (1, 2)
Optimal policy for n_in_queue =  18: (1, 2)
Optimal policy for n_in_queue =  19: (1, 2)
Optimal policy for n_in_queue =  20: (1, 2)
Optimal policy for n_in_queue =  21: (1, 2)
Optimal policy for n_in_queue =  22: (1, 2)
Optimal policy for n_in_queue =  23: (1, 2